In [7]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import asyncio
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
open_router_key = os.getenv("OPENROUTER_API_KEY")

In [10]:
model_client = OpenAIChatCompletionClient(
    model='gemini-1.5-flash',
    api_key=api_key,
)

## Serializing Termination Conditions

In [5]:
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination

text_mention_termination = TextMentionTermination('STOP')
max_message_termination = MaxMessageTermination(5)

termination_condition = text_mention_termination & max_message_termination

termination_config =  termination_condition.dump_component()

termination_config.model_dump_json()

'{"provider":"autogen_agentchat.base.AndTerminationCondition","component_type":"termination","version":1,"component_version":1,"description":null,"label":"AndTerminationCondition","config":{"conditions":[{"provider":"autogen_agentchat.conditions.TextMentionTermination","component_type":"termination","version":1,"component_version":1,"description":"Terminate the conversation if a specific text is mentioned.","label":"TextMentionTermination","config":{"text":"STOP"}},{"provider":"autogen_agentchat.conditions.MaxMessageTermination","component_type":"termination","version":1,"component_version":1,"description":"Terminate the conversation after a maximum number of messages have been exchanged.","label":"MaxMessageTermination","config":{"max_messages":5,"include_agent_event":false}}]}}'

In [6]:
final_termination_condition = termination_condition.load_component(termination_config)

## Serializing Agents

In [11]:
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    description="An assistant agent that can answer questions and perform tasks.",
    system_message="You are a helpful assistant to answer the users query.",
)

user_proxy = UserProxyAgent(name="user_proxy")

In [12]:
agent.dump_component()
user_proxy.dump_component()

ComponentModel(provider='autogen_agentchat.agents.UserProxyAgent', component_type='agent', version=1, component_version=1, description='An agent that can represent a human user through an input function.', label='UserProxyAgent', config={'name': 'user_proxy', 'description': 'A human user'})

In [ ]:
import json

with open('agent_config.json', 'w') as f:
    json.dump(agent.dump_component().model_dump(), f, indent=2)

with open('user_proxy_config.json', 'w') as f:
    json.dump(user_proxy.dump_component().model_dump(), f, indent=2)

In [16]:
with open("user_proxy_config.json", "r") as f:
    user_proxy_config = json.load(f)

user_proxy = user_proxy.load_component(user_proxy_config)

## Serializing Teams

In [17]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants=[agent, user_proxy],
    termination_condition=final_termination_condition,
)

In [18]:
from pydantic import SecretStr

team_config = team.dump_component()

def secret_str_serializer(obj):
    if isinstance(obj, SecretStr):
        return obj.get_secret_value()
    raise TypeError(f"Type {type(obj)} not serializable")

with open('team.json', 'w') as f:
    json.dump(team_config.model_dump(), f, indent=2, default=secret_str_serializer)